In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import talib as ta
import os
import mplfinance as mpf

In [2]:
def get_time_stamp():
    return datetime.now().strftime('%Y%m%d%H%M%S')

In [3]:
directory = 'tokocrypto-' + get_time_stamp()
os.makedirs(directory, exist_ok=True)

In [4]:
def get_pairs() :
    return requests.get('https://www.tokocrypto.com/open/v1/common/symbols').json()

In [5]:
def get_ticker(symbol, interval) :
    return requests.get('https://api.binance.com/api/v1/klines?symbol=' + symbol + '&interval=' + interval + '&limit=1000').json()

In [6]:
width_ratio = 0.2

In [7]:
# pairs = get_pairs()
# symbols = [item['symbol'] for item in pairs['data']['list'] if 'symbol' in item and item['symbol'].endswith('USDT') and item['symbol'] != 'BTC_USDT' and item['type'] == 3]
# print(symbols)

In [21]:
pairs = get_pairs()
symbols = [item['symbol'] for item in pairs['data']['list'] if 'symbol' in item and item['symbol'].endswith('USDT') and item['type'] == 1]
remove_list = ['BTC_USDT', 'USDC_USDT', 'USDP_USDT', 'USTC_USDT', 'FDUSD_USDT', 'TUSD_USDT', 'AEUR_USDT']
symbols = [item for item in symbols if item not in remove_list]
print(symbols)

['1000SATS_USDT', '1INCH_USDT', '1MBABYDOGE_USDT', 'AAVE_USDT', 'ACA_USDT', 'ACE_USDT', 'ACH_USDT', 'ACM_USDT', 'ACT_USDT', 'ACX_USDT', 'ADA_USDT', 'ADX_USDT', 'AEVO_USDT', 'AGLD_USDT', 'AI_USDT', 'ALCX_USDT', 'ALGO_USDT', 'ALICE_USDT', 'ALPACA_USDT', 'ALPHA_USDT', 'ALPINE_USDT', 'ALT_USDT', 'AMP_USDT', 'ANKR_USDT', 'APE_USDT', 'API3_USDT', 'APT_USDT', 'ARB_USDT', 'ARDR_USDT', 'ARKM_USDT', 'ARK_USDT', 'ARPA_USDT', 'AR_USDT', 'ASR_USDT', 'ASTR_USDT', 'ATA_USDT', 'ATM_USDT', 'ATOM_USDT', 'AUCTION_USDT', 'AUDIO_USDT', 'AVAX_USDT', 'AVA_USDT', 'AXL_USDT', 'AXS_USDT', 'BADGER_USDT', 'BAKE_USDT', 'BAL_USDT', 'BANANA_USDT', 'BAND_USDT', 'BAR_USDT', 'BAT_USDT', 'BB_USDT', 'BCH_USDT', 'BEAMX_USDT', 'BEL_USDT', 'BETA_USDT', 'BICO_USDT', 'BIFI_USDT', 'BLUR_USDT', 'BNB_USDT', 'BNT_USDT', 'BOME_USDT', 'BONK_USDT', 'BSW_USDT', 'BTTC_USDT', 'C98_USDT', 'CAKE_USDT', 'CATI_USDT', 'CELO_USDT', 'CELR_USDT', 'CETUS_USDT', 'CFX_USDT', 'CGPT_USDT', 'CHESS_USDT', 'CHR_USDT', 'CHZ_USDT', 'CITY_USDT', 'CKB_USD

In [9]:
# pairs = get_pairs()
# symbols = [item['symbol'] for item in pairs['data']['list'] if 'symbol' in item and item['symbol'].endswith('IDR') and item['symbol'] != 'BTC_IDR']
# print(symbols)

In [10]:
def get_df_from_ticker(ticker_name, interval) :
    ticker = get_ticker(ticker_name + 'USDT', interval)
    df_1 = pd.DataFrame(ticker, columns=[
    'Open time',
    'Open',
    'High',
    'Low',
    'Close',
    'Volume',
    'Close time',
    'Quote asset volume',
    'Number of trades',
    'Taker buy base asset volume',
    'Taker buy quote asset volume',
    'Ignore'])
    df_1['Date'] = pd.to_datetime(df_1['Close time'], unit='ms')
    df_1['Open'] = pd.to_numeric(df_1['Open'], errors='coerce')
    df_1['Close'] = pd.to_numeric(df_1['Close'], errors='coerce')
    df_1['High'] = pd.to_numeric(df_1['High'], errors='coerce')
    df_1['Low'] = pd.to_numeric(df_1['Low'], errors='coerce')
    df_1['Volume'] = pd.to_numeric(df_1['Volume'], errors='coerce')
    df_1 = df_1.sort_values(by='Date')
    return df_1

In [11]:
dfBtcUsdt = get_df_from_ticker('BTC', '1d')

In [12]:
def get_history_btc_pair(tickername, df_coin_usdt, df_btc_usdt) :
    ticker_name_low = tickername.lower()
    df_merged = pd.merge(df_coin_usdt, df_btc_usdt, on='Date', suffixes=('_' + ticker_name_low, '_btc'))
    df_merged['Open_' + ticker_name_low +'_btc'] = df_merged['Open_' + ticker_name_low] / df_merged['Open_btc']
    df_merged['High_' + ticker_name_low +'_btc'] = df_merged['High_' + ticker_name_low] / df_merged['High_btc']
    df_merged['Low_' + ticker_name_low + '_btc'] = df_merged['Low_' + ticker_name_low] / df_merged['Low_btc']
    df_merged['Close_'+ ticker_name_low +'_btc'] = df_merged['Close_' + ticker_name_low] / df_merged['Close_btc']
    df_merged['Volume_'+ ticker_name_low +'_btc'] = df_merged['Volume_' + ticker_name_low]

    df_final = df_merged[['Date', 'Open_' + ticker_name_low +'_btc', 'High_' + ticker_name_low +'_btc', 'Low_' + ticker_name_low + '_btc', 'Close_' + ticker_name_low +'_btc', 'Volume_' + ticker_name_low +'_btc']].copy()
    df_final.rename(columns={
        'Open_' + ticker_name_low +'_btc': 'Open',
        'High_' + ticker_name_low +'_btc': 'High',
        'Low_' + ticker_name_low + '_btc': 'Low',
        'Close_' + ticker_name_low +'_btc': 'Close',
        'Volume_' + ticker_name_low +'_btc': 'Volume'
    }, inplace=True)

    df_final['RSI_14'] = ta.RSI(df_final['Close'], timeperiod=14)

    df_final['OBV'] = ta.OBV(df_final['Close'], df_final['Volume'])

    slowk, slowd = ta.STOCH(df_final['High'], df_final['Low'], df_final['Close'],
                           fastk_period=5,
                           slowk_period=3,
                           slowd_period=3)
    df_final['Stoch_%K'] = slowk
    df_final['Stoch_%D'] = slowd

    min_time_period = min(len(df_merged['Close_' + ticker_name_low])-1, 90)
    df_final['Beta'] = ta.BETA(df_merged['Close_' + ticker_name_low], df_merged['Close_btc'], timeperiod=min_time_period)
    df_final['Correl'] = ta.CORREL(df_merged['Close_' + ticker_name_low], df_merged['Close_btc'], timeperiod=min_time_period)

    df_final['Change1D'] = df_final['Close'].pct_change() * 100
    df_final['Change7D'] = df_final['Close'].pct_change(periods=7) * 100
    df_final['Change30D'] = df_final['Close'].pct_change(periods=30) * 100

    # df_final['EMA_13'] = df_final['Close'].ewm(span=13, adjust=False).mean()
    # df_final['EMA_21'] = df_final['Close'].ewm(span=21, adjust=False).mean()

    return df_final

In [13]:
def show_candlestick(tickername, df) :
    df.set_index('Date', inplace=True)
    mpf.plot(df, type='candle', style='binance', volume=True, title=f'Pergerakan Harga {tickername} terhadap BTC',
         savefig=dict(fname=os.path.join(directory, 'Candlestick-' + tickername + '-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight'))

In [14]:
def show_chart(df, tickername) :
    time_now = datetime.now().strftime('%d-%m-%Y %H:%M:%S')

    # Membuat figure dan dua subplot yang berbagi sumbu x
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, ncols=1, sharex=True, figsize=(14, 10))
    # fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(14, 10))

    # Pergerakan Harga
    ax1.plot(df['Date'], df['Close'], label=f'{tickername}/BTC', color='blue')
    # ax1.plot(df['Date'], df['EMA_13'], linewidth=0.5, label='EMA 13', color='yellow')
    # ax1.plot(df['Date'], df['EMA_21'], linewidth=0.5, label='EMA 21', color='orange')
    ax1.set_title(f'Pergerakan Harga {tickername} terhadap BTC')
    ax1.set_ylabel('Harga (BTC)')
    ax1.legend()
    ax1.grid(True)
    ax1.text(0.99, 1.05, time_now,
         transform=ax1.transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    ax1.text(0.5, 0.5, 'Generated by Yoga',
         transform=ax1.transAxes,
         fontsize=20, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)

    # RSI
    ax2.plot(df['Date'], df['RSI_14'], label=f'RSI {tickername}/BTC', color='orange')
    ax2.axhline(70, linestyle='--', alpha=0.5, color='red')  # Garis overbought
    ax2.axhline(30, linestyle='--', alpha=0.5, color='green')  # Garis oversold
    ax2.set_title(f'Grafik RSI {tickername} terhadap BTC')
    ax2.set_xlabel('Tanggal')
    ax2.set_ylabel('RSI')
    ax2.legend()
    ax2.grid(True)
    ax2.text(0.99, 1.05, time_now,
         transform=ax2.transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    ax2.text(0.5, 0.5, 'Generated by Yoga',
         transform=ax2.transAxes,
         fontsize=20, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)

    # Stochastic
    ax3.plot(df['Date'], df['Stoch_%K'], label='%K', color='blue')
    ax3.plot(df['Date'], df['Stoch_%D'], label='%D', color='orange')
    ax3.axhline(80, linestyle='--', alpha=0.5, color='red')  # Garis overbought
    ax3.axhline(20, linestyle='--', alpha=0.5, color='green')  # Garis oversold
    ax3.set_title(f'Grafik Stochastic {tickername} terhadap BTC')
    ax3.set_xlabel('Tanggal')
    ax3.set_ylabel('Nilai')
    ax3.legend()
    ax3.grid(True)
    ax3.text(0.99, 1.05, time_now,
         transform=ax3.transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    ax3.text(0.5, 0.5, 'Generated by Yoga',
         transform=ax3.transAxes,
         fontsize=20, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)

    # OBV
    ax4.plot(df['Date'], df['OBV'], label='OBV', color='blue')
    ax4.set_title(f'Grafik OBV {tickername}')
    ax4.set_xlabel('Tanggal')
    ax4.set_ylabel('Nilai')
    ax4.legend()
    ax4.grid(True)
    ax4.text(0.99, 1.05, time_now,
         transform=ax4.transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    ax4.text(0.5, 0.5, 'Generated by Yoga',
         transform=ax4.transAxes,
         fontsize=20, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)

    # Menyesuaikan tata letak agar tidak saling tumpang tindih
    plt.tight_layout()
    # plt.show()
    plt.savefig(os.path.join(directory, 'Chart-' + tickername + '-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

    show_candlestick(tickername, df)

In [15]:
def show_chart_rsi(df) :
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['RSI_14'], color='blue')
    plt.axhline(70, linestyle='--', alpha=0.5, color='red', label='Overbought (70)')
    plt.axhline(30, linestyle='--', alpha=0.5, color='green', label='Oversold (30)')
    plt.title('Nilai RSI Pair BTC')
    plt.xlabel('Ticker')
    plt.ylabel('RSI')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'RSI-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [16]:
def show_chart_beta(df) :
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Beta'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='magenta', label='Beta = 0')
    plt.axhline(1, linestyle='--', alpha=0.5, color='cyan', label='Beta = 1')
    plt.title('Nilai Beta (Volatilitas Harga Token Terhadap BTC)')
    plt.xlabel('Ticker')
    plt.ylabel('Beta')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Beta-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [17]:
def show_chart_correl(df) :
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Correl'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='magenta', label='Correlation = 0')
    plt.axhline(-1, linestyle='--', alpha=0.5, color='cyan', label='Correlation = -1')
    plt.axhline(1, linestyle='--', alpha=0.5, color='cyan', label='Correlation = 1')
    plt.title('Nilai Pearson Correlation Coefficient Terhadap BTC')
    plt.xlabel('Ticker')
    plt.ylabel('Correlation')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Correlation-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [18]:
def show_chart_change(df, chart_title) :
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='red', label='Change = 0%')
    plt.title(chart_title)
    plt.xlabel('Ticker')
    plt.ylabel('Perubahan (%)')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, chart_title + '-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [19]:
def show_multiple_chart(tickers, df_btc_usdt, show_individual_chart=False) :
    rsi_list = []
    beta_list = []
    correl_list = []
    latest_change_1d_list = []
    latest_change_7d_list = []
    latest_change_30d_list = []
    tickers.sort()

    for i, ticker in enumerate(tickers) :
        print('Processing ' + ticker + '...')
        will_show = False
        description = ''
        df_coin_idr_1 = get_df_from_ticker(ticker, '1d')
        df_btc_pair = get_history_btc_pair(ticker, df_coin_idr_1, df_btc_usdt)
        if show_individual_chart :
            show_chart(df_btc_pair, ticker)

        latest_rsi = df_btc_pair['RSI_14'].iloc[-1]
        rsi_list.append({'Label': ticker, 'RSI_14': latest_rsi})

        if latest_rsi < 30 :
            will_show = True
            description = 'Low RSI'
        if latest_rsi > 70 :
            will_show = True
            description = 'High RSI'

        latest_beta = df_btc_pair['Beta'].iloc[-1]
        beta_list.append({'Label': ticker, 'Beta': latest_beta})

        if latest_beta < 0 :
            will_show = True
            description = 'Negative Beta'

        latest_correl = df_btc_pair['Correl'].iloc[-1]
        correl_list.append({'Label': ticker, 'Correl': latest_correl})

        if latest_correl < 0 :
            will_show = True
            description = 'Negative Correlation'

        # if will_show and not show_individual_chart :
            # print(description)
            # show_chart(df_btc_pair, ticker)

        latest_change_1d = df_btc_pair['Change1D'].iloc[-1]
        latest_change_7d = df_btc_pair['Change7D'].iloc[-1]
        latest_change_30d = df_btc_pair['Change30D'].iloc[-1]

        if not pd.isna(latest_change_1d) :
            latest_change_1d_list.append({'Label': ticker, 'Data': latest_change_1d})
        if not pd.isna(latest_change_7d) :
            latest_change_7d_list.append({'Label': ticker, 'Data': latest_change_7d})
        if not pd.isna(latest_change_30d) :
            latest_change_30d_list.append({'Label': ticker, 'Data': latest_change_30d})

    rsi_list = sorted(rsi_list, key=lambda x: x['RSI_14'])
    rsi_df = pd.DataFrame(rsi_list)
    show_chart_rsi(rsi_df)

    beta_list = sorted(beta_list, key=lambda x: x['Beta'])
    beta_df = pd.DataFrame(beta_list)
    show_chart_beta(beta_df)

    correl_list = sorted(correl_list, key=lambda x: x['Correl'])
    corr_df = pd.DataFrame(correl_list)
    show_chart_correl(corr_df)

    latest_change_1d_list = sorted(latest_change_1d_list, key=lambda x: x['Data'])
    latest_change_1d_list_df = pd.DataFrame(latest_change_1d_list)
    show_chart_change(latest_change_1d_list_df, 'Perubahan 1 Hari Terhadap BTC')

    latest_change_7d_list = sorted(latest_change_7d_list, key=lambda x: x['Data'])
    latest_change_7d_list_df = pd.DataFrame(latest_change_7d_list)
    show_chart_change(latest_change_7d_list_df, 'Perubahan 7 Hari Terhadap BTC')

    latest_change_30d_list = sorted(latest_change_30d_list, key=lambda x: x['Data'])
    latest_change_30d_list_df = pd.DataFrame(latest_change_30d_list)
    show_chart_change(latest_change_30d_list_df, 'Perubahan 30 Hari Terhadap BTC')

    excel_file = os.path.join(directory, 'price_tko-' + get_time_stamp() + '.xlsx')
    with pd.ExcelWriter(excel_file) as writer:
        rsi_df.to_excel(writer, sheet_name='RSI', index=False)
        beta_df.to_excel(writer, sheet_name='Beta', index=False)
        corr_df.to_excel(writer, sheet_name='Correlation', index=False)
        latest_change_1d_list_df.to_excel(writer, sheet_name='Change 1D', index=False)
        latest_change_7d_list_df.to_excel(writer, sheet_name='Change 7D', index=False)
        latest_change_30d_list_df.to_excel(writer, sheet_name='Change 30D', index=False)


In [20]:
tickers = [symbol.removesuffix('_USDT') for symbol in symbols]
# print(tickers)

show_multiple_chart(tickers, dfBtcUsdt, True)

Processing 1000SATS...
Processing 1INCH...


/Users/yogabprawira/PycharmProjects/crypto-price/.venv/lib/python3.13/site-packages/mplfinance/_arg_validators.py:84: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+


Processing 1MBABYDOGE...
Processing AAVE...


/Users/yogabprawira/PycharmProjects/crypto-price/.venv/lib/python3.13/site-packages/mplfinance/_arg_validators.py:84: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+


Processing ACA...


...

/Users/yogabprawira/PycharmProjects/crypto-price/.venv/lib/python3.13/site-packages/mplfinance/_arg_validators.py:84: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+


Processing ZIL...


/Users/yogabprawira/PycharmProjects/crypto-price/.venv/lib/python3.13/site-packages/mplfinance/_arg_validators.py:84: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+


Processing ZRX...


/Users/yogabprawira/PycharmProjects/crypto-price/.venv/lib/python3.13/site-packages/mplfinance/_arg_validators.py:84: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+
